In [1]:
import json
import sys
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
import tqdm
import wandb
from ignite.contrib.handlers import wandb_logger
from ignite.engine import (Engine, Events, create_supervised_evaluator,
                           create_supervised_trainer)
from ignite.handlers import ModelCheckpoint
from ignite.handlers.param_scheduler import LRScheduler
from ignite.metrics import Accuracy, Loss
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from torch import nn
from torch.functional import F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

sys.path.append('../')
from attacks.deepfool import deepfool
from src.datasets import FordDataset
from src.models import TransformerClassification
from src.utils import build_optimizer, str2torch

In [2]:
with open('../configs/transformer_87.json') as f:
    config =  json.load(f)
config['train']['optimizer'] = str2torch(config['train']['optimizer'])

In [3]:
test_path = "../data/FordA/FordA_TEST.arff"
test_dataset = FordDataset(test_path, config['data'])
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TransformerClassification(config).to(device)
weights = torch.load('../models/trans_2outp_87.pth')
model.load_state_dict(weights)

<All keys matched successfully>

In [5]:
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)
        return y_pred, y

test_evaluator = Engine(validation_step)

# Attach metrics to the evaluators
metrics = {
    'accuracy': Accuracy(output_transform=lambda x: (torch.argmax(x[0], dim=1), x[1])),
}

for name, metric in metrics.items():
    metric.attach(test_evaluator, name)

In [6]:
test_evaluator.run(test_dataloader)
test_evaluator.state.metrics['accuracy']

c:\Users\ptmeg\miniconda3\envs\ts_robustness\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


0.8696969696969697

In [7]:
iters = []
for i in tqdm.tqdm(range(len(test_dataset))):
    test_sample = torch.from_numpy(test_dataset[i][0]).unsqueeze(1)
    r_tot, loop_i, label, k_i, pert_image = deepfool(test_sample, model, 2, max_iter=30)
    iters.append(loop_i)

  8%|▊         | 874/10560 [00:12<02:39, 60.86it/s]c:\Users\ptmeg\OneDrive\Документы\Skoltech\Term3\ML\ts_robustness\notebooks\..\attacks\deepfool.py:65: RuntimeWarning: divide by zero encountered in scalar divide
  pert_k = abs(f_k)/np.linalg.norm(w_k.flatten())
100%|██████████| 10560/10560 [02:33<00:00, 68.77it/s]


In [21]:
unique, counts = np.unique(iters, return_counts=True)

In [24]:
np.mean(iters), np.std(iters)

(2.4644886363636362, 1.576901306655389)